# Mocap Data Pre-processing

In [1]:
using LinearAlgebra, Random
using StatsBase, Statistics
using Distributions, MultivariateStats   # Categorical, PCA
using Quaternions    # For manipulating 3D Geometry
using MeshCat        # For web visualisation / animation
using PyPlot         # Plotting
using AxUtil, Flux   # Optimisation

# small utils libraries
using ProgressMeter, Formatting, ArgCheck
using DelimitedFiles, NPZ, BSON

In [2]:
DIR_MOCAP_MTDS = "." #"""../../../mocap-mtds/";   # different cos I'm in dev folder

# Data loading and transformation utils
include(joinpath(DIR_MOCAP_MTDS, "io.jl"))

# MeshCat skeleton visualisation tools
include(joinpath(DIR_MOCAP_MTDS, "mocap_viz.jl"))

In [3]:
cmu_loco = readdlm("../data/mocap/cmu/cmu_locomotion_lkp.txt", '\t')[:,1];
database = "../data/mocap/holden/cmu"
files_cmu = [joinpath(database, f * ".bvh") for f in cmu_loco]
files_cmu = collect(filter(x-> isfile(x) && x !== "rest.bvh", files_cmu));

database = "../data/mocap/edin-style-transfer/"
files_edin = [joinpath(database, f) for f in readdir(database)];

### Read in example data

In [4]:
proc = mocapio.process_file(files_cmu[40]);   
# 54 = PACING/STOP, 40 = BACKWARD, 115=TRIUMPHANT, 190=TWISTY, not 198 (JUMP)!

In [11]:
include(joinpath(DIR_MOCAP_MTDS, "io.jl"))

In [6]:
fps = 60

In [12]:
?mocapio.construct_inputs

In [8]:
yy = mocapio.construct_outputs(proc, fps=fps);

In [10]:
false && @showprogress for (_i, ii) in enumerate(range(1,stop=size(u,1), step=2))
    PyPlot.cla()
    tmp_new = hcat(u[ii,1:12], u[ii,13:24])
    plot(tmp_new[:,1], tmp_new[:,2])
    scatter(0,0)
    # arrow(0,0, diff(tmp_new[7:8,1])[1], diff(tmp_new[7:8,2])[1], width=.2)
    for j in 1:11
        dirΔ = [tmp_new[j+1,1] - tmp_new[j,1], tmp_new[j+1,2] - tmp_new[j,2]]
        dirΔ /= sqrt(sum(z->z^2, dirΔ))
        std_cθ, std_sθ = add_angle(u[ii,24+j], u[ii,36+j], 0, -1)
        cθ, sθ = add_angle(dirΔ[1], dirΔ[2], std_cθ, -std_sθ)
        @assert isapprox(reim(complex(0, 1) * complex(u[ii,24+j], -u[ii,36+j]) * complex(dirΔ[1], dirΔ[2])) |> collect, 
            [cθ, sθ])
        arrow(tmp_new[j,1],tmp_new[j,2], cθ, sθ, width=.05)
    end

#     _tmp_dir = reim(-complex(1, 0) *complex(u[ii,24+8], -u[ii,36+8]) )  # u[34+ii,8], u[34+ii,12+8]
#     @assert isapprox(collect(_tmp_dir), [-u[ii,24+8], u[ii,36+8]])
#     arrow(0, 0, _tmp_dir[1],_tmp_dir[2], width=0.2/sqrt(sum(diff([gca().get_xlim()...]))), 
#     color=ColorMap("tab10")(1))
#     plot(zeros(2), [-u[ii,36+8], u[ii,36+8]]*1.5, color="k", linewidth=0.4)

    gca().set_xlim(-20, 20)
    # gca().set_ylim(-15, 15)
    gca().set_aspect("equal")
    savefig(format("img/traj_{:03d}.png", _i))
end

In [11]:
plot(atan.(u[ii-50:ii+50, 24+7], u[ii-50:ii+50, 36+7]))
gcf().set_size_inches(3,2)

In [12]:
tmp = mocapio.process_file(files_edin[1], fps=fps);

## Getting ready for training

In [13]:
# Load data from raw BVH files (1-2 mins), or load a saved version
DATA_FROM_SAVED = false
fps = 30

if DATA_FROM_SAVED
    Xs = BSON.load("edin_Xs.bson")[:Xs];
    Ys = BSON.load("edin_Ys.bson")[:Ys];
else
    proc_files = map(files_edin) do f
        mocapio.process_file(f, fps=fps);
    end;
    println("loaded files..."); flush(stdout)
    Xs = map(proc_files) do proc
        convert(Matrix{Float32}, mocapio.construct_inputs(proc; include_ftcontact=true, 
                include_ftmid=true, joint_pos=false, fps=fps, speed=true))
    end;
    println("Xs finished processing..."); flush(stdout)
    Ys = map(proc_files) do proc
        convert(Matrix{Float32}, 
            mocapio.construct_outputs(proc; include_ftcontact=false, fps=fps))
    end;
    println("Ys finished processing..."); 
    
    println("Do you want to save? (y/n)")
    
    for i = 1:10
        userinput = uppercase(chomp(readline()))[1]
        if userinput == 'Y'
            println("SAVING..."); flush(stdout)
            BSON.bson("edin_Xs_30fps_new.bson", Xs=Xs)
            BSON.bson("edin_Ys_30fps_new.bson", Ys=Ys)
            break
        elseif userinput == 'N'
            break
        end
    end
end;

Ysraw = Ys
Xsraw = Xs;

In [7]:
if !(@isdefined vis) 
    # Create a new visualizer instance (MeshCat.jl)
    vis = Visualizer()
    open(vis)
end
vis = mocapviz.create_animation([mocapio.reconstruct_modelled(Ysraw[1])[1:200,:,:]], 
    "test"; vis=vis, linemesh=mocapviz.yellowmesh, camera=:back)

### CMU #91

In [32]:
database91 = "../data/mocap/cmu/misc_retarget_simple/"
files_cmu91 = [joinpath(database91, f) for f in readdir(database91) if f[end-2:end] == "bvh"];
rm_ixs = [6,15,16]  # 6 too short, 15-16 unusual and very difficult
files_cmu91 = [f for (i, f) in enumerate(files_cmu91[1:36]) if !(i in rm_ixs)];

In [84]:
# Load data from raw BVH files (1-2 mins), or load a saved version
fps = 30

proc_files = map(files_cmu91) do f
    mocapio.process_file(f, fps=fps);
end;
println("loaded files..."); flush(stdout)
Xs = map(proc_files) do proc
    convert(Matrix{Float32}, mocapio.construct_inputs(proc; include_ftcontact=true, 
            include_ftmid=true, joint_pos=false, fps=fps, speed=true))
end;
Xs = [x[46:end,:] for x in Xs];   # start: first couple of seconds standing, so rm
println("Xs finished processing..."); flush(stdout)
Ys = map(proc_files) do proc
    convert(Matrix{Float32}, 
        mocapio.construct_outputs(proc; include_ftcontact=false, fps=fps))
end;
Ys = [x[46:end,:] for x in Ys];   # start: first couple of seconds standing, so rm
println("Ys finished processing..."); 

println("Do you want to save? (y/n)")

for i = 1:10
    userinput = uppercase(chomp(readline()))[1]
    if userinput == 'Y'
        println("SAVING..."); flush(stdout)
        BSON.bson("cmu91_Xs_30fps.bson", Xs=Xs)
        BSON.bson("cmu91_Ys_30fps.bson", Ys=Ys)
        break
    elseif userinput == 'N'
        break
    end
end

Ysraw = Ys
Xsraw = Xs;

In [86]:
# Load CMU91 data
DA_Usraw = BSON.load("cmu91_Xs_30fps.bson")[:Xs]
DA_Ysraw = BSON.load("cmu91_Ys_30fps.bson")[:Ys];

In [87]:
# Load Mason data
mason_Usraw = BSON.load("edin_Xs_30fps.bson")[:Xs];
mason_Ysraw = BSON.load("edin_Ys_30fps.bson")[:Ys];

In [90]:
# Save concatenation
BSON.bson("edin_Xs_30fps_DA.bson", Xs=vcat(mason_Usraw, DA_Usraw))
BSON.bson("edin_Ys_30fps_DA.bson", Ys=vcat(mason_Ysraw, DA_Ysraw));

#### Create new dataset for pytorch

In [91]:
mason_Ysraw = [y[2:end,:] for y in mason_Ysraw]
mason_Usraw = [hcat(u[2:end,1:end-8], u[1:end-1,end-7:end]) for u in mason_Usraw];

In [9]:
Ysraw = [y[2:end,:] for y in Ysraw]
Usraw = [hcat(u[2:end,1:end-8], u[1:end-1,end-7:end]) for u in Xsraw];

In [22]:
Ysraw = vcat(mason_Ysraw, Ysraw);
Usraw = vcat(mason_Usraw, Usraw);

In [10]:
include("util.jl")

In [11]:
# Standardise inputs and outputs
standardize_Y = fit(mocaputil.MyStandardScaler, reduce(vcat, Ysraw),  1)
standardize_U = fit(mocaputil.MyStandardScaler, reduce(vcat, Usraw),  1)

Ys = [mocaputil.scale_transform(standardize_Y, y[2:end, :] ) for y in Ysraw];  # (1-step ahead of u)
Us = [mocaputil.scale_transform(standardize_U, u[1:end-1,:]) for u in Usraw];  # (1-step behind y)

@assert (let c=cor(Usraw[1][1:end-1, :], Ysraw[1][2:end, :], dims=1); 
        !isapprox(maximum(abs.(c[.!isnan.(c)])), 1.0); end) "some input features perfectly correlated"

# to invert: `mocaputil.invert(standardize_Y, y)`

In [99]:
npzwrite("./data/edin_Ys_30fps_DA.npz", Dict(string(i)=>Ys[i] for i in 1:length(Ys)))

In [98]:
npzwrite("./data/edin_Us_30fps_DA.npz", Dict(string(i)=>Us[i] for i in 1:length(Us)))

# Ramer-Douglas-Peucker
from Rosetta Code

In [12]:
# source https://rosettacode.org/wiki/Ramer-Douglas-Peucker_line_simplification
const Point = Vector{Float64}
 
function perpdist(pt::Point, lnstart::Point, lnend::Point)
    d = normalize!(lnend .- lnstart)
 
    pv = pt .- lnstart
    # Get dot product (project pv onto normalized direction)
    pvdot = dot(d, pv)
    # Scale line direction vector
    ds = pvdot .* d
    # Subtract this from pv
    return norm(pv .- ds)
end
 
function rdp(plist::Vector{Point}, ϵ::Float64 = 1.0)
    if length(plist) < 2
        throw(ArgumentError("not enough points to simplify"))
    end
 
    # Find the point with the maximum distance from line between start and end
    distances  = collect(perpdist(pt, plist[1], plist[end]) for pt in plist)
    dmax, imax = findmax(distances)
 
    # If max distance is greater than epsilon, recursively simplify
    if dmax > ϵ
        fstline = plist[1:imax]
        lstline = plist[imax:end]
 
        recrst1 = rdp(fstline, ϵ)
        recrst2 = rdp(lstline, ϵ)
 
        out = vcat(recrst1, recrst2)
    else
        out = [plist[1], plist[end]]
    end
    
    return unique(out)
end

rdp(X::AbstractMatrix, ϵ = 1.0) = reduce(hcat, rdp([X[i,:] for i in 1:size(X,1)], ϵ))'

## Smooth trajectory

In [13]:
function merge_knots!(u; take_first=true, merge_triplets=true)
    i = 1
    while i < length(u)-2
        if u[i] == 1
            if merge_triplets && u[i+2] > 0
                u[i], u[i+1], u[i+2] = 0, 1, 0
                i += 3
                continue
            elseif u[i+1] > 0
                if take_first
                    u[i+1] = 0
                else
                    u[i] = 0
                end
                i += 2
                continue
            end
        end
        i += 1
    end
    return u
end
merge_knots(u; take_first=true, merge_triplets=true) = merge_knots!(copy(u); 
    take_first=take_first, merge_triplets=merge_triplets)

function reinforce_knots(u)
    n = length(u)
    out = copy(u)
    for i in 1:n
        if u[i] == 1
            i > 3 && (out[i-3] = 1)
            i < n - 2 && (out[i+3] = 1)
        end
    end
    return out
end

In [14]:
# defaults (see get below: 0.7, 0.3 respectively)
epsilons = Dict(10=>0.6, 14=>0.5, 15=>0.9, 16=>0.9, 17=>0.5, 18=>1.2, 19=>1.2, 20=>1.2, 21=>1.2, 25=>1.2, 
    26=>2.5, 27=>2.5, 28=>3.5, 31=>1.5)
θ2d_thrsh = Dict(10=>0.28, 18=>0.25, 19=>0.25, 26=>Inf, 27=>0.4);
addl_pts = Dict(10 => [462], 18 => [1570], 19=>[880,920, 1022], 22=>[510], 26=>[946, 2890])
rm_pts = Dict(10=>81:88, 18=>vcat(670:700, 1250:1290), 19=>887:895, 20=>1293:1314, 21=>vcat(659:661, 1724:1732),
        22=>509:512, 23=>991:998, 24=>1040:1058, 25=>vcat(41:47, 146:170, 590:600, 800:820,1440:1460, 1720:1740),
        27=>vcat(69:74, 284:290, 2060:2070), 28=>vcat(2543:2560, 3448:3460), 
        28=>vcat(3829:3835, 3940:3960, 4065:4080),29=>vcat(2427:2440), 30=>vcat(257:270, 1120:1130),
        31=>vcat(962:980, 1060:1080, 1200:1210, 1322:1330, 1532:1540, 1600:1630, 2096:2107))
addl_sgl = Dict(3 => [60], 20=>[1293, 1314, 1330], 21=>[650,660,670,1710,1728], 23=>[980,1010], 
    24=>[650,1040,1060,1210,1220,1230,1240], 25=>[30, 50, 1440, 1460, 1490], 26=>[1945, 1970, 2860, 2920],
    27=>[70, 100, 1530], 28=>[3455], 30=>[264], 31=>vcat([60,276], [1060, 1070, 1090, 1100])) 
addl_discont = Dict(28=>[3455], 30=>[264])

In [822]:
ff = 4
trj = mocapio._traj_fk(zeros(Float64, 3), proc_files[ff][:, 2], proc_files[ff][:, 3], proc_files[ff][:, 1]);

In [823]:
ϵ = get(epsilons, ff, 0.7)  # for early ones, 0.7 looks good; but up to 2.5 for sexy style
ix_rng = 1:length(proc_files[ff][:,1])    # neccesary ∵ bug atm in mocapio which triples length of output
_traj_mat = hcat(trj[1][ix_rng], trj[2][ix_rng])

# Do RDP transform and extract knots
ts = let x = rdp(_traj_mat, ϵ)
    map(1:size(x,1)) do i
        findfirst((x[i,1] .≈ _traj_mat[:,1]) .& (x[i,2] .≈ _traj_mat[:,2]))
    end
end;

In [824]:
# Find additional turning points by looking at large large second "derivatives" (differences)
_x, _z = proc_files[ff][:, 2], proc_files[ff][:, 3]
θ = atan.(_x, _z);
knots = abs.(diff(vcat(0, θ))) .> get(θ2d_thrsh, ff, 0.3);

# add/remove custom points specified above
rm_knots = vcat(1, get(rm_pts, ff, []))   # always rm initial knot
knots[rm_knots] .= 0
addl_knots = get(addl_pts, ff, [])
length(addl_knots) > 0 && (knots[addl_knots] .= 1)

# reinforce these addl knots and merge with RDP knots
knots = reinforce_knots(knots)
extreme_turns = vcat(findall(merge_knots(knots; take_first=false, merge_triplets=false)), get(addl_sgl, ff, []))
knots[ts] .= 1
addl_knots = get(addl_sgl, ff, [])
length(addl_knots) > 0 && (knots[addl_knots] .= 1)
ts = findall(merge_knots(knots; take_first=false, merge_triplets=false));
ts = sort(vcat(ts, get(addl_discont, ff, [])));

In [825]:
# filter(i->i in 1:200, findall(knots))

In [826]:
# Calculate spline coefficients via OLS
# spl = AxUtil.Math.BSpline(3, (ts .-1) ./ (ix_rng[end] -1))
# splBasis = AxUtil.Math.basis_eval(spl, (ix_rng .- 1)./(ix_rng[end] -1))
knots = (ts .-1) ./ (ix_rng[end] -1)
eval_ts = (ix_rng .- 1)./(ix_rng[end] -1)

knots = ts .- 1
eval_ts = ix_rng .- 1
splBasis = AxUtil.Math.bsplineM(eval_ts, knots, 3+1)
derivBasis = AxUtil.Math.bsplineM(eval_ts, knots, 3+1, 1)
deriv2Basis = AxUtil.Math.bsplineM(eval_ts, knots, 3+1, 2)
A = _traj_mat' / splBasis';
_smth_trj = splBasis * A';
_smth_deriv = derivBasis * A';
_smth_deriv2 = deriv2Basis * A';

In [827]:
function fix_atan_jumps(x) 
    offset = 0
    out = similar(x); out[1] = x[1]
    for i in 2:length(x)
        if x[i] - x[i-1] > π
            offset -= 2π
        elseif x[i] - x[i-1] < -π
            offset += 2π
        end
        out[i] = x[i] + offset
    end
    return out
end

atan_dt(y, x, ẏ, ẋ) = (ẏ * x - ẋ * y) / (x^2  + y^2)
atan_d2t(y, x, ẏ, ẋ, ÿ, ẍ) = let num=(ẏ * x - ẋ * y); denom=(x^2  + y^2); num_dt=(ÿ * x - ẍ * y)
    denom_dt= 2*(x*ẋ  + y*ẏ); (num_dt*denom - num*denom_dt) / denom^2; end

In [62]:
include(joinpath(DIR_MOCAP_MTDS, "io.jl"))

In [68]:
?mocapio.construct_outputs

In [67]:
proc_files[1]

In [64]:
plot(tmp[1:500,end])
plot(tmp[1:500,end-1])

In [63]:
tmp = mocapio.construct_inputs(proc_files[2]; joint_pos=false, fps=fps, speed=false, root_angle=false, 
    direction=:none, lead_ft=true, turn360mask=true)

In [833]:
body_angle = mocapio.construct_inputs(proc_files[ff]; joint_pos=false, fps=fps, speed=false)[:,[13,19]]

x = fix_atan_jumps(atan.(body_angle[:, 1], body_angle[:, 2]))
pwise = rdp(hcat(1:length(body_angle[:,1]), x), 3.0)
ixs = pwise[:,1]
pi_mult = round.(Int, x[Int.(ixs)] / 2π)
ix_change = findall(diff(pi_mult) .!= 0) .+ 1
ix_change = ixs[hcat(ix_change .- 1, ix_change)'[:]]

plot(x)
[gca().axvline(i, linestyle=":", color="grey") for i in ix_change]
title(format("{:d} changepoints", Int(length(ix_change) / 2)))
display(ix_change)

In [829]:
_ix_rng = (31:430) .+ 0
fig, axs = subplots(1,3,figsize=(10,3))
axs[1].plot(fix_atan_jumps(atan.(diff(_smth_trj[_ix_rng,2]), diff(_smth_trj[_ix_rng,1]))))
[axs[1].axhline(θ, color="grey", linestyle=":") for θ in [π/2, π, 3π/2]]
d = atan_dt.(diff(_smth_trj[_ix_rng,2]), diff(_smth_trj[_ix_rng,1]), 
        diff(_smth_deriv[_ix_rng,2]), diff(_smth_deriv[_ix_rng,1]))
axs[2].plot(d)
axs[2].plot(diff(fix_atan_jumps(atan.(diff(_smth_trj[_ix_rng,2]), diff(_smth_trj[_ix_rng,1])))))
[axs[2].axhline(thrsh, color="grey", linestyle=":") for thrsh in [-0.15, 0.15]]

breaches = abs.(d) .> 0.15
if length(breaches) > 1
    d2 = atan_d2t.(diff(_smth_trj[_ix_rng,2]), diff(_smth_trj[_ix_rng,1]), 
            diff(_smth_deriv[_ix_rng,2]), diff(_smth_deriv[_ix_rng,1]),
            diff(_smth_deriv2[_ix_rng,2]), diff(_smth_deriv2[_ix_rng,1]))
    breach_tp = findall(sign.(d2[breaches][2:end]) .!= sign.(d2[breaches][1:end-1]))
    breach_tp = findall(breaches)[breach_tp]
    [axs[2].axvline(p, color="grey", linestyle=":") for p in breach_tp]
#     axs[3].plot(d2)
end

turn_window = 10
for ix in findall(breaches)   # smoosh...
    breaches[max(0, ix-turn_window):min(end, ix+turn_window)] .= true
end

body_angle = mocapio.construct_inputs(proc_files[ff]; joint_pos=false, fps=fps, speed=false)[:,[13,19]]
axs[3].plot(fix_atan_jumps(atan.(body_angle[_ix_rng .- 30,1], body_angle[_ix_rng .- 30,2])))
highlight = atan.(body_angle[_ix_rng .- 30,1], body_angle[_ix_rng .- 30,2])
highlight[.!vcat(false, breaches)] .= NaN
axs[3].plot(fix_atan_jumps(highlight))


In [830]:
# plot(trj[1][1:400], trj[2][1:400]); gca().set_aspect("equal")
plot(_smth_trj[_ix_rng, 1], _smth_trj[_ix_rng, 2]); gca().set_aspect("equal")
scatter(_smth_trj[_ix_rng[100:100:400], 1], _smth_trj[_ix_rng[100:100:400], 2], marker="x");
scatter(_smth_trj[_ix_rng[vcat(false,breaches)], 1], _smth_trj[_ix_rng[vcat(false,breaches)], 2], 
    marker="o", alpha=0.4); 
# scatter(_smth_trj[_ix_rng[breach_tp], 1], _smth_trj[_ix_rng[breach_tp], 2], marker="o", alpha=0.4); 
gca().set_aspect("equal")

In [565]:
num_plots = ceil(Int, ix_rng[end] / 400)
fig, axs = subplots(num_plots, 1, figsize=(7, 5*num_plots))
for i in 1:num_plots
    ix_plot = ((i-1)*400 +1):min(400*i-1, ix_rng[end])
    axs[i].plot(trj[1][ix_plot], trj[2][ix_plot])
    axs[i].plot(_smth_trj[ix_plot,1], _smth_trj[ix_plot,2])
    extr = filter(t->t in ix_plot, extreme_turns)
    for j in extr; axs[i].scatter(trj[1][j], trj[2][j], marker="x", c="k"); end
end

In [112]:
ix_plot = 1:200
plot(trj[1][ix_plot], trj[2][ix_plot])
let x = rdp(_traj_mat, ϵ); ixs=[t for (i,t) in enumerate(ts) if t in ix_plot]; plot(trj[1][ixs], trj[2][ixs]); end

In [1698]:
ff = 27
u = mocapio.construct_inputs(proc_files[ff], joint_pos=false, fps=30)

In [1699]:
plot(atan.(u[:,13], u[:,19]))

## Possibly re-do foot contacts as only initial touchdown

In [838]:
using HMMBase

In [839]:
hmm = HMM([0.9 0.1; 0.1 0.9], [DiscreteNonParametric([0,1], [0.9,0.1]), DiscreteNonParametric([0,1], [0.2,0.8])])

In [901]:
_ii = 25
_u = viterbi(hmm, Usraw[_ii][:,end-6]) .- 1;

In [844]:
using AxPlot

In [842]:
cumsum(vcat(0, [size(Ys[i], 1) ÷ 64 for i in 32:64]...))

In [902]:
offset = 0
# plot(Usraw[2][(1:256) .+ offset,end-5])
fig, axs = subplots(2,1)
axs[1].plot(_u[(1:512) .+ offset] .+ 0.02)
axs[2].plot(Usraw[_ii][(1:512) .+ offset,end-6] .+ 0.04)
AxPlot.rmaxislabel_x(axs[1]); tight_layout()

### Need to standardize inputs and outputs

`MLPreprocessing.jl` contains something like a port of `StandardScaler` from `sklearn`. However, as of 18/05/2019 it appears to be essentially broken, and given its broad generality to lots of array types, I don't want to get into this whole thing. So I'm just going to do define something myself.

In [9]:
?MyStandardScaler

In [5]:
include(joinpath(DIR_MOCAP_MTDS, "util.jl"))
import .mocaputil: MyStandardScaler, scale_transform
# const scale_transform = mocaputil.transform

In [6]:
standardize_Y = fit(MyStandardScaler, reduce(vcat, Ysraw),  1)
standardize_X = fit(MyStandardScaler, reduce(vcat, Xsraw),  1)

Ys = [scale_transform(standardize_Y, y) for y in Ysraw];
Xs = [scale_transform(standardize_X, x) for x in Xsraw];

note that we can reconstruct the original data via the command:

    invert(standardize_Y, y)
    invert(standardize_X, x)
    
in the relevant array comprehensions.

### It is worth checking to see if it makes sense to scale everything
As shown in the input/output columns below, there is nothing that appears problematic in doing this

In [2]:
?mocapio.process_file

In [12]:
?mocapio.construct_inputs

In [10]:
?mocapio.construct_outputs

In [140]:
Ys[1][:,65:end]

In [167]:
?mocaputil.MyStandardScalar

In [159]:
mutable struct OutputDifferencer{T}
    first_frame::Array{T, 1}
    operate_on::AbstractArray{L where L <: Int,1}
end

Base.length(s::OutputDifferencer) = length(s.first_frame, 1)
Base.copy(s::OutputDifferencer) = OutputDifferencer(copy(s.first_frame), copy(s.operate_on))

function StatsBase.fit(::Type{OutputDifferencer}, Y)
    @argcheck size(Y, 2) in [64, 68]
    return OutputDifferencer(Y[1,:], 4:64)
end

function difference_transform(s::OutputDifferencer, Y)
    @assert Y[1,:] == s.first_frame
    return hcat(Y[2:end,1:(s.operate_on[1] - 1)], 
        diff(Y[:,s.operate_on], dims=1),
        Y[2:end,(s.operate_on[end] + 1):end])
end

function fit_transform(::Type{OutputDifferencer}, Y)
    s = fit(OutputDifferencer, Y)
    return s, difference_transform(s, Y)
end

function invert(s::OutputDifferencer, Y)
    tr_first = reshape(s.first_frame, 1, :)
    inv = cumsum(vcat(tr_first[:,s.operate_on], Y[:,s.operate_on]), dims=1)
    return vcat(tr_first,
                hcat(Y[:,1:(s.operate_on[1] - 1)], 
                    inv[2:end,:],
                    Y[:,(s.operate_on[end] + 1):end])
                )
end

In [144]:
s, Ydiff = fit_transform(OutputDifferencer, Ys[1])

In [149]:
imshow(Ys[1][1:200,:]); gca().set_aspect("auto")

In [166]:
all(isapprox.(invert(s, Ydiff), Ys[1], atol=1e-5))

## Last checks with visualisation

In order to visualise the *direction* of ±60 frames, we have to essentially perform FK since these directions are all relative to the forward direction at the current location. I've done this using fairly vanilla trigonometric functions below as my knowledge of quaternions and manipulating 3D graphics is not as good as it could be.

In [161]:
function angle_to_z_axis(v)
    cθ, sθ = mocapio._trigvecs(reshape(v, 1, 2), reshape([0f0 1f0], 1, 2))
    atan(-sθ[1], cθ[1])
end
rotationm(θ) = [cos(θ) sin(θ); -sin(θ) cos(θ)]

function add_angle(c1, s1, c2, s2)
    θ₁, θ₂ = atan(s1,c1), atan(s2, c2)
    return cos(θ₁ + θ₂), sin(θ₁ + θ₂)
end

In [1203]:
Complex(-1/sqrt(2), -1/sqrt(2)) * Complex(+1/sqrt(2), -1/sqrt(2)+0.05)

In [ ]:
atan()

In [1190]:
methods(angle)

In [ ]:
mutable struct MyAngle
    

In [114]:
ii = 436

In [112]:
plot(atan.(Xsraw[4][ii-50:ii+50, 24+7], Xsraw[4][ii-50:ii+50, 36+7]))
gcf().set_size_inches(3,2)

In [1205]:
angle(Complex(0, -1))

In [116]:
tmp_new = hcat(Xsraw[4][ii,1:12], Xsraw[4][ii,13:24])
plot(tmp_new[:,1], tmp_new[:,2])
scatter(0,0)
arrow(0,0, diff(tmp_new[7:8,1])[1], diff(tmp_new[7:8,2])[1], width=.2)
for j in 1:11
    dirΔ = [tmp_new[j+1,1] - tmp_new[j,1], tmp_new[j+1,2] - tmp_new[j,2]]
    dirΔ /= sqrt(sum(z->z^2, dirΔ))
    cθ, sθ = Complex(0, -1) * Complex(Xsraw[4][ii,24+j], Xsraw[4][ii,36+j]) * Complex(dirΔ[1], dirΔ[2])
    
#     std_cθ, std_sθ = add_angle(Xsraw[4][ii,24+j], Xsraw[4][ii,36+j], 0, -1)
#     cθ, sθ = add_angle(dirΔ[1], dirΔ[2], std_cθ, -std_sθ)
    arrow(tmp_new[j,1],tmp_new[j,2], cθ, sθ, width=.05)
end
gca().set_aspect("equal")
# ii += 5

In [108]:
if !(@isdefined vis) 
    # Create a new visualizer instance (MeshCat.jl)
    vis = Visualizer()
    open(vis)
end
vis = mocapviz.create_animation([mocapio.reconstruct_modelled(Ysraw[4])[300:550,:,:]], 
    "test"; vis=vis, linemesh=mocapviz.yellowmesh, camera=:back)

## Everything looks good

**Note** the input traces look slightly odd during sharp/stopping "about" turns as frequently the trajectory turns in the opposite direction to the body. Since the turning circle is so tight, it is probably not well defined which way the trajectory turns, and anchoring to the *actual position* of the root joint is a good a way as any. The point is that while it may look a little odd, the processing is doing the right thing, and the information will be available to the model.

## Look at principal components

In [218]:
?mocapio.construct_inputs

In [ ]:
parent = data[parents,:]
data = data[2:end, :]  # remove root (has no line)

In [70]:
function plot_2d_skeleton(joints::AbstractMatrix)
    @argcheck size(joints) == (21, 3)
    parents=[1,2,3,4,1,6,7,8,1,10,11,12,12,14,15,16,12,18,19,20]
    scatter(joints[:,1], joints[:,2]);
    [plot([[joints[p,j], joints[e+1,j]] for j in 1:2]..., c="k") for (e,p) in enumerate(parents)]
    [text(joints[i,1]+0.2, joints[i,2]-0.7, i) for i in 1:21]
    gca().set_aspect("equal")
    gca().axis("off");
end

function plot_2d_skeleton(root_y::AbstractFloat, joints::AbstractMatrix)
    @argcheck size(joints) == (20, 3)
    joints = vcat([0 root_y 0], joints); 
    plot_2d_skeleton(joints)
end

In [112]:
plot_2d_skeleton(Ys[1][4,4], reshape(Ys[1][4,5:end], 3, 20)')
savefig("joints.pdf")

In [80]:
using TexTables

In [78]:
summarize(randn(30,4))

In [89]:
[TableCol("test", collect(1:10), randn(10)) TableCol("test2", collect(1:10), randn(10))]

In [102]:
function latex_table(x::AbstractMatrix, headers::AbstractVector{T}) where T <: Union{String, Real}
    n,d = size(x)
    reduce(hcat, [TableCol(string(headers[i]), collect(1:n), x[:,i]) for i in 1:d])
end
latex_table(x::AbstractMatrix) = latex_table(x, collect(1:size(x,2)))

In [103]:
latex_table(randn(3,4))

In [111]:
@fmt Real = "{:.2f}"
let joints = reshape(Ys[1][4,5:end], 3, 20)'; 
    joints = vcat([0 Ys[1][4,4] 0], joints); 
end |> x->round.(x, digits=2) |> x->latex_table(x, ["x","y","z"]) |> to_tex |> print
@fmt Real = "{:.3f}"

In [121]:
allE = reduce(vcat, Ys);
allE = convert(Matrix{Float32}, allE);

In [124]:
zsc(x, dims) = (x .- mean(x, dims=dims)) ./ std(x, dims=dims)

pc_all = fit(PCA, zsc(allE[:,4:63], 1)', pratio=0.999)

varexpl = cumsum(principalvars(pc_all))/tvar(pc_all)
bd=findfirst.([varexpl .> x for x in [0.9,0.95,0.99]])
plot(1:length(varexpl), varexpl)
gca().axhline(1, linestyle=":")
for b in bd
    plot([b,b], [varexpl[1], varexpl[b]], color=ColorMap("tab10")(7), linestyle=":")
    plot([.5, b], [varexpl[b], varexpl[b]], color=ColorMap("tab10")(7), linestyle=":")
end
gca().set_xlim(0.5,25.5); gca().set_ylim(varexpl[1],1.025);
gcf().set_size_inches(3,2)

#### Note
This is different (worse, i.e. need more PCs) than the graph I showed to Chris. This is because I was in the Eulerian frame, where direction(s) of movement probably captured most of PCs. Removing these high variance directions, common to all joints makes it much harder.